### FILE UPLOAD TO SNOWFLAKE USING SQLALCHEMY

In [1]:
from sqlalchemy import create_engine, text

##### Functions to create snowflake connection, internal staging, creating table, staging data and loading staged data to Snowflake database

In [2]:
def create_snowflake_connection(user, password, account, warehouse, database, schema):
    connection_string = f'snowflake://{user}:{password}@{account}/' \
                        f'?warehouse={warehouse}&database={database}&schema={schema}'
    return connection_string

In [3]:
def create_internal_stage(engine, stage_name):
    create_stage_query = f"""
    CREATE STAGE IF NOT EXISTS {stage_name};
    """
    with engine.connect() as connection:
        connection.execute(text(create_stage_query))

In [4]:
def create_table_with_csv_structure(engine, table_name):
    create_table_query = f"""
    CREATE OR REPLACE TABLE {table_name} (
        id INT,
        topic_name STRING,
        year STRING,
        level STRING,
        introduction STRING,
        learning_outcome STRING,
        summary_page_link STRING,
        pdf_file_Link STRING
    );
    """
    with engine.connect() as connection:
        connection.execute(text(create_table_query))

In [5]:
def put_data_into_stage(engine, csv_file_path, stage_name):
    put_data_query = f"""
    PUT file://{csv_file_path} @{stage_name};
    """
    with engine.connect() as connection:
        connection.execute(text(put_data_query))


In [6]:
def load_data_from_stage_to_table(engine, table_name, stage_name):
    copy_into_query = f"""
    COPY INTO {table_name} FROM @{stage_name} FILE_FORMAT = (TYPE = CSV SKIP_HEADER = 1) ON_ERROR = 'CONTINUE';
    """
    with engine.connect() as connection:
        connection.execute(text(copy_into_query))

In [13]:
import configparser
config = configparser.ConfigParser()
config.read('../configuration.properties')
# config.get('SNOWFLAKE', )

snowflake_user = config['SNOWFLAKE']['User']
snowflake_password = config['SNOWFLAKE']['Password']
snowflake_account = config['SNOWFLAKE']['Account']
snowflake_warehouse = config['SNOWFLAKE']['Warehouse']
snowflake_database = config['SNOWFLAKE']['Database']
snowflake_schema = config['SNOWFLAKE']['Schema']

# Create a connection string
connection_string = create_snowflake_connection(snowflake_user, snowflake_password, snowflake_account,
                                                snowflake_warehouse, snowflake_database, snowflake_schema)


Step 1: Create an engine for Snowflake connection

In [14]:
engine = create_engine(connection_string)

Step 2: Create an internal stage in Snowflake

In [15]:
stage_name = 'my_stage'
create_internal_stage(engine, stage_name)


Step 3: Create a table with reference to CSV structure

In [18]:
table_name = 'refresher_readings'
csv_file_path = '../web-scraping-and-dataset/scraped_data.csv'
create_table_with_csv_structure(engine, table_name)

Step 4: Put data into stage

In [19]:
put_data_into_stage(engine, csv_file_path, stage_name)

Step 5: Load data from stage to table using COPY INTO command

In [20]:
load_data_from_stage_to_table(engine, table_name, stage_name)